# A5

In [1]:
# Standard Imports
import numpy as np
import matplotlib.pylab as plt

# Q1: Electrophysiology Experiment

## Helper functions

In [2]:
def MonkeyFixationSpikes(x, T, pop):
    """
    spike_times = MonkeyFixationSpikes(x, T, pop)

    Given the gaze direction, outputs the spiking activity of the monkey's
    neurons.

    Input:
      x is the monkey's gaze direction
      T is the duration of the fixation (in seconds)
      pop is a 6xN matrix of LIF parameters

    Output:
      spike_times is an list containing N arrays of time-stamps indicating
        when each neuron fired.  
    """
    dt = 0.001  # time-step size for simulations
    xts = x*np.ones(int(T/dt))
    sp, v = Stim2Spikes(xts, dt, pop, interp=True)
    return sp

def Stim2Spikes(x, dt, lif, interp=False):
    '''
    spike_times, V = Stim2Spikes(x, dt, LIFparams, interp=False)

    Given the input current, outputs the spiking activity of a
    population neurons.

    Input:
      J is a PxN array of input currents, with each row containing the
        input current for N neurons at a given time
      dt is the time step (in seconds)
      lif is a (5+K)xN matrix of LIF parameters
      interp determines whether spike times should be interpolated
          (False is the default)

    Output:
      spike_times is an array of time-stamps indicating when the neurons
        fired.  In particular,
      V is a PxN array of membrane potentials
    '''

    P = np.shape(x)[0] # number of x-values
    N = np.shape(lif)[1] # number of neurons

    T = dt * P # seconds
    t = np.array(range(P+1)) * dt  # time steps

    #J_th = lif[0,:]
    #tau_RC = lif[1,:]
    #tau_ref = lif[2,:]
    alpha = lif[3,:]
    Jbias = lif[4,:]
    enc = lif[5,:]

    V = np.array( np.zeros(N) )
    Vrec = np.array( np.zeros([P+1,N]) )
    Vrec[0,:] = V

    J = np.array( np.zeros([P,N]) ) # to store input currents
    #pdb.set_trace()  # this sets a breakpoint

    for k in range(1,P):
        for m in range(N):
            J[k,m] = alpha[m]*x[k-1]*enc[m] + Jbias[m]
        # next m
    # next k

    # Pass the work onto the integrator
    spike_times, Vrec = Current2Spikes(J, dt, lif, interp)

    return spike_times, Vrec

def Current2Spikes(J, dt, lif=np.array([[1.,0.02, 0.002, 1, 0, -1]]).T, interp=False):
    '''
    spike_times, V = Current2Spikes(J, dt, lif, interp=False)

    Given the input current, outputs the spiking activity of a
    population neurons.

    Input:
      J is a PxN array of input currents, with each row containing the
        input current for N neurons at a given time
      dt is the time step (in seconds)
      lif is a (5+K)xN matrix of LIF parameters
      interp determines whether spike times should be interpolated
          (False is the default)

    Output:
      spike_times is an array of time-stamps indicating when the neurons
        fired.  In particular,
      V is a PxN array of membrane potentials
    '''
    P = np.shape(J)[0] # number of x-values
    N = np.shape(lif)[1] # number of neurons

    T = dt * P # seconds
    t = np.array(range(P)) * dt  # time steps

    J_th = lif[0,:]
    tau_RC = lif[1,:]
    tau_ref = lif[2,:]
    #alpha = lif[3,:]
    #Jbias = lif[4,:]
    #enc = lif[5,:]

    V = np.array( np.zeros(N) )
    Vrec = np.array( np.zeros([P,N]) )
    Vrec[0,:] = V

    max_number_spikes = int(np.floor(T/min(tau_ref)))
    spike_times_matrix = np.zeros([max_number_spikes,N])
    spike_count = np.zeros(N, dtype=int)
    refracting = np.zeros(N)


    for k in range(1,P):
        for m in range(N):
            J_M = J[k-1,m]
            dV = (-1.0 / tau_RC[m]) * (V[m]-J_M * J_th[m])

            #active = (t[k] >= refracting)
            Vn = 0  # default if still in refraction
            if t[k]>=refracting[m]:
                Vo = V[m] # previous V
                if abs(t[k]-refracting[m])>=dt:
                    Vn = Vo + dV * dt # new V
                else:
                    Vn = Vo + dV * (t[k] - refracting[m])

                #Vn = clip(Vn,0,1.e20)

                if Vn>=1.0:
                    if interp:
                        # Linear interpolation of time for threshold crossing
                        tstar = ( t[k-1]*(Vn-1) - t[k]*(Vo-1) ) / (Vn-Vo)
                    else:
                        # Choose t[k] as spike time
                        tstar = t[k]

                    spike_times_matrix[spike_count[m],m] = tstar
                    spike_count[m] = spike_count[m] + 1
                    refracting[m] = tstar + tau_ref[m]
                    Vn = 0.
                # end of if

            # end of refraction if
            V[m] = Vn

        # next m

        #pdb.set_trace()  # this sets a breakpoint
        Vrec[k,:] = V

    # next k

    spike_times = []
    for n in range(N):
        spike_times.append(spike_times_matrix[0:spike_count[n],n])

    return spike_times, Vrec


def PlotSpikeRaster(st, y_range=[0, 1.]):
    '''
    PlotSpikeRaster(spiketimes, y_range=[0, 1.])

    Plots a spike raster plot for a list of arrays of spike times.

    Input:
      spiketimes is a list of arrays of spike times, like that returned
          by the function Stim2Spikes.
      y_range is a 2-tuple that holds the y-values that the raster ticks
          should be drawn between
    '''
    N = len(st)  # number of neurons

    levels = np.linspace(y_range[0], y_range[1], N+1, endpoint=True)
    plt.figure(figsize=(10,4))
    for n in range(N):
        nspikes = len(st[n])
        y = [ [levels[n]]*nspikes , [levels[n+1]]*nspikes ]
        #y = y_range[0] + [levels[n]]*nspikes
        plt.vlines(st[n], levels[n], levels[n+1], color=np.random.rand(3))
        #plt.plot(vstack((st[n],st[n])), y, color=random.rand(3))
    plt.ylim(y_range)
    return

def CountSpikes(st, tstart, tend):
    '''
    counts = CountSpikes(st, tstart, tend)

    Counts how many spikes occur between the start and end times.

    Input:
      st is a list of arrays of spike times, like that returned
          by the function Stim2Spikes. That is, st[0] is an array of spike
          times for the first neuron, st[1] is for the next neuron, etc.

    Output:
      counts is an array of integers indicating how many spikes each
          neuron had.
    '''
    N = len(st)
    r = np.zeros(N)
    for n in range(N):
        for s in st[n]:
            if (tstart<=s and s<=tend):
                r[n] += 1
    return r


In [3]:
#=====================
# CHOOSE A MONKEY
# The monkey's are numbered 0, 1, 2, 3, and 4.
# This will read in the LIF parameters for the monkey's neurons
# as well as the spiking trains for the unknown sequence.
#=====================
infile = open('monkey3.npz', 'rb')
varsin = np.load(infile, allow_pickle=True)
pop = varsin['pop']   # parameters for population of neurons
Asp = varsin['Asp']   # spike trains for unknown sequence
N = len(pop[1])  # number of neurons

## (a) Behavioural Sampling Experiment

In [4]:
# Select a spread of stimulus values from 0 to 9. Choose at least 100 samples.
# pick num = 901. values are 0,0.01,0.02,...



X = np.linspace(0.,9.,num=500)
ft = 0.4
A = np.zeros((500,20))
row = 0

for x in X:
    spike_times = MonkeyFixationSpikes(x,ft,pop)
    col = 0
    for st in spike_times:
        if len(st) != 0:
            fr = 1/(st[1:] - st[:len(st)-1])
        else:
            fr=0
        A[row][col] = np.mean(fr)
        col+=1
    row += 1
# ***** YOUR CODE HERE *****

## (b) View Tuning Curves

In [5]:
# ***** YOUR CODE HERE *****
for i in range(20):
    plt.plot(values, A[:,i])
plt.title('Tuning Curves')
plt.xlabel('Gaze Direction')
plt.ylabel('Firing Rate (Hz)')

NameError: name 'values' is not defined

## (c) Compute the Decoding Weights

In [ ]:
# ***** YOUR CODE HERE *****
# SVD u @ np.diag(s) @ vh
U,Sigma,VTrans = np.linalg.svd(A,full_matrices =False)
Sigma = np.diag(Sigma)
D = VTrans.T@np.linalg.inv(Sigma)@U.T@X

## (d) View Spike Raster of Unknown Sequence

In [ ]:
# ***** YOUR CODE HERE *****
PlotSpikeRaster(Asp,y_range = [0.0,1.0])

## (e) Decode the Unknown Code

In [ ]:
# Spike trains for unknown sequence are stored in 'Asp'
# Asp is a list of arrays of spike trains, one array per neuron.

In [ ]:
# Compute firing rates
delta_t = 0.5
fr = np.zeros((5,20))
i = 0
tstart = 0.
tend = 0.5
for i in range(5):
    spikeTrain = CountSpikes(Asp,tstart,tend)
    fr[i,:] = spikeTrain/delta_t
    tstart = tend
    tend += 0.5
# ***** YOUR CODE HERE *****

In [ ]:
# Decode unknown code
unknown_code = np.around(fr@D)

# ***** YOUR CODE HERE *****

## (f) Display the Unknown Code

In [ ]:
# ***** YOUR CODE HERE *****
fig = plt.figure(figsize=(8,6))
fixation = np.linspace(0.5,2.5,num=5)
plt.plot(fixation,unknown_code,'gd')
plt.xlabel('fixation duration in seconds')
plt.ylabel('gaze direction')
plt.grid('on')

# Q2: LSTM

In [ ]:
# You may include some Python code to help you.
import numpy as np
Wf = np.array([0,10,0,0])
bf = -5
Wi = np.array([0,0,8,0])
bi = -4
Wo = np.array([0,0,0,9])
bo = -4.5
Wc = np.array([1,0,0,0])
bc = 0
ht_1 = np.array([-0.03])
Ct_1 = np.array([0.05])

def sigma(x):
    deno = 1+np.exp(-x)
    return 1/deno

def gate(w,v,b):
    return sigma(w@v+b)

## (a)

### (i)

In [ ]:

# You can include some code, if you want.
x1 = np.array([1,0,0])
v1 = np.hstack((ht_1,x1))
print(v1)
ft = gate(Wf,v1,bf)
it = gate(Wi,v1,bi)
ot = gate(Wo,v1,bo)
Ctildet = np.tanh(Wc@v1+bc)
Ct = ft * np.array([0.05]) + it * Ctildet
ht = ot * np.tanh(Ct)
print("values of ft gate:", ft)
print("values of it gate:", it)
print("values of ot gate:", ot)
print("value of Ct is: ", Ct[0])
print("value of new ht is: ",ht[0])

$$h_{t-1} = [-0.03]$$
$$x1 = [1,0,0]$$
$$v_1 = [-0.03,1,0,0]$$
$$f_t = 0.9933071490757153\approx 1$$
$$i_t = 0.01798620996209156 \approx 0$$
$$o_t = 0.01098694263059318\approx 0$$
$$C_t = f_t \odot C_{t-1} + i_t \odot \tilde{C_t} \approx 1\odot C_{t-1} + 0 \odot  \tilde{C_t} = 0.05$$

$$h_t = o_t \odot tanh(C_t) \approx 0 \odot tanh(C_t) = 0$$


### (ii)

In [ ]:
x2 = np.array([0,1,0])
v2 = np.hstack((ht_1,x2))
print(v2)
ft = gate(Wf,v2,bf)
it = gate(Wi,v2,bi)
ot = gate(Wo,v2,bo)
Ctildet = np.tanh(Wc@v2+bc)
Ct = ft * np.array([0.05]) + it * Ctildet
ht = ot * np.tanh(Ct)
print("values of ft gate:", ft)
print("values of it gate:", it)
print("values of ot gate:", ot)
print("value of Ct is: ", Ct[0])
print("value of new ht is: ",ht[0])

$$h_{t-1} = [-0.03]$$
$$x2 = [0,1,0]$$
$$v_2 = [-0.03,0,1,0]$$
$$f_t = 0.0066928509242848554 \approx 0$$
$$i_t = 0.9820137900379085 \approx 1$$
$$o_t = 0.01098694263059318 \approx 0$$
$$C_t = f_t \odot C_{t-1} + i_t \odot \tilde{C_t} \approx 0\odot C_{t-1} + 1 \odot  \tilde{C_t} = 0.05$$

$$h_t = o_t \odot tanh(C_t) \approx 0 \odot tanh(C_t) = 0$$


### (iii)

In [ ]:
x3 = np.array([1,0,1])
v3 = np.hstack((ht_1,x3))
print(v3)
ft = gate(Wf,v3,bf)
it = gate(Wi,v3,bi)
ot = gate(Wo,v3,bo)
Ctildet = np.tanh(Wc@v3+bc)
Ct = ft * np.array([0.05]) + it * Ctildet
ht = ot * np.tanh(Ct)
print("values of ft gate:", ft)
print("values of it gate:", it)
print("values of ot gate:", ot)
print("value of Ct is: ", Ct[0])
print("value of new ht is: ",ht[0])

$$h_{t-1} = [-0.03]$$
$$x3 = [0,1,0]$$
$$v_3 = [-0.03,1,0,1]$$
$$f_t = 0.9933071490757153 \approx 1$$
$$i_t = 0.01798620996209156 \approx 0$$
$$o_t = 0.9890130573694068 \approx 1$$
$$C_t = f_t \odot C_{t-1} + i_t \odot \tilde{C_t} \approx 1 \odot C_{t-1} + 0 \odot  \tilde{C_t} = 0.05$$

$$h_t = o_t \odot tanh(C_t) \approx 1 \odot tanh(C_t) = 0.049958375$$

For i, ii and iii, the outputs from calculation and approximation are similiar.
We see that the value of gate have an impact on new $C_t$ and $h_t$. If the gate is open( value are close to 1), the gate is closed, then the old values will be ignored.

For example, if $f_t$ and/or $i_t$ is nearly zero, the $C_t$ will be independent of $C_{t-1}$ and/or  $\tilde{C_t}$
Similarly, if $o_t$ is zero, the $h_t$ will be independent of $C_t$ and will have a valuie of $0$.


## (b)

In [ ]:
# set x_t to be [1 1 0] reasons shown below
xt = np.array([1,1,0])
# let C_t-1 to be 0.04 this time and h_t-1 = [-0.03] as well
vt = np.array([-0.03,1,1,0])
ft = sigma(Wf @ vt + bf)
it = sigma(Wi @ vt + bi)
ot = sigma(Wo @ vt + bo)
Ctildet = np.tanh(Wc @ vt + bc)
Ct = ft * np.array([0.05]) + it * Ctildet
ht = ot * np.tanh(Ct)
gates = np.array([ft,it,ot])
print("values of gates f_t, i_t and o_t are: " + str(gates))
print("calculated value of C_t is: " + str(Ct[0]))
print("value of new C_t is: " + "C_t-1 + h_t-1 = " + str(0.05+(-0.03)))
print(Ctildet)

In [ ]:
x = np.array([1,1,0])
v = np.hstack((ht_1,x))
print(v)
ft = gate(Wf,v,bf)
it = gate(Wi,v,bi)
ot = gate(Wo,v,bo)
Ctildet = np.tanh(Wc @ v + bc)
print(Ctildet)
Ct = ft * np.array([0.05]) + it * Ctildet
ht = ot * np.tanh(Ct)
print("values of ft gate:", ft)
print("values of it gate:", it)
print("values of ot gate:", ot)
print("value of Ct is: ", Ct[0])
print("value of new ht is: ",ht[0])
print("value of new C_t is: C_t-1 + h_t-1 = ", str(0.05+(-0.03)))

In order to approximate the sum of your old cell state and your hidden state, the gate should be open, which means the gate of $f_t$ and $i_t$ must be open. 
Therefore we can pick x to be [1,1,0] or [1,1,1]. Applying sigmoid function to large values will open the gate( $\approx$ 1) and the gate will be 'open'. If the $f_t$ and $i_t$ have values close to 0, new cell keeps both the state of old cell and hidded state.

## (c)

In [ ]:
x = np.array([1/2,1/2,0])
v = np.hstack((ht_1,x))
print(v)
ft = gate(Wf,v,bf)
it = gate(Wi,v,bi)
ot = gate(Wo,v,bo)
Ctildet = np.tanh(Wc @ v + bc)
print(Ctildet)
Ct = ft * np.array([0.05]) + it * Ctildet
ht = ot * np.tanh(Ct)
print("values of ft gate:", ft)
print("values of it gate:", it)
print("values of ot gate:", ot)
print("value of Ct is: ", Ct[0])
print("value of new ht is: ",ht[0])
print("value of new ht is : 1/2(C_t-1 + h_t-1) = ", 1/2*(0.05+(-0.03)))

If we want the $h_t$ to be the average of previous cell state and input value of $h_{t-1}$, we want the value of $f_t$ and $i_t$ to be 0.5. 
We can derive from the equations, such as:
$$h_t = o_t \odot tanh(C_t) \approx  o_t \odot C_t $$
$$h_t = o_t \odot \left(f_t \odot C_{t-1} + i_t \odot \tilde{C_t} \right)$$
$$\tilde{C_t} = tanh(W_Cv_t+b_C)$$
therefore, 
$$ h_t= o_t \odot \left(f_t \odot C_{t-1} + i_t \odot (W_Cv_t+b_C)\right)$$
$$ h_t= o_t \odot \left(f_t \odot C_{t-1} + i_t \odot h_{t-1}\right)$$



$$f_t = 0.5 \\ i_t =0.5$$
$$Sigmoid(W_f@V +b_f) == 0.5 \\ and \\ Sigmoid(W_i@V +b_i) == 0.5$$
which is 
$$W_f@V +b_f == 0  \\ and \\W_i@V +b_i == 0$$
Solve for V we need v to be [-0.03,1/2,1/2, some value]
-0.03 is fixed because $h_{t-1} = -0.03$ and last value in v is not restrited by solving these equations.